#### Recursive Descent Parsing
CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
February 2023
***

Consider the following LL(1) Grammar
* $E \rightarrow N \ | \ OEE$
* $O \rightarrow +\ |\ *$ 
* $N \rightarrow 0\ |\ 1\ |\ 2\ |\ 3$

The starting non-terminal is $E$

The first sets are
* $FIRST(OEE) = FIRST(O) = \{+,*\}$
* $FIRST(N) = \{0,1,2,3\}$

***

Let's construct a class for the recursive descent parser (RDP). There will be a function for each nonterminal and a function to test whether the current input character what the grammar expects. We'll call this testing function ___current_input_matches_target_.

***

In [12]:
class RDP:  # RDP stands for recursive descent parser
    def __init__(self) -> None:
        ###################################
        # Tuple defining an LL(1) grammar #
        # • set of nonterminals           #
        # • set of terminals              #
        # • starting nonterminal          #
        # • set of productions            #
        ###################################
        self.nonterminals: set[str] = {'E', 'O', 'N'}  # set of nonterminals. Each nonterminal will have its own function
        self.starting_nonterminal = self.E  # Starting nonterminal
        self.terminals: set[str] = {'+', '*', '0', '1', '2', '3'}  # set of terminals
        # Productions                               # Defined within the nonterminal functions

        ##########################################
        # Define FIRST sets for each nonterminal #
        ##########################################
        self.first: dict[str, set[str]] = dict()
        self.first['O'] = {'+', '-'}
        self.first['N'] = {'0', '1', '2', '3'}

        ###########################################
        # Variables for Managing the input string #
        ###########################################
        self.input = None
        self.num_chars_read: int = 0
    def parse_input(self, input_: str) -> None:
        """ Call this function from main."""
        self.input = input_
        self.starting_nonterminal()  # Run the RDP by calling the starting nonterminal
        if self.num_chars_read == len(self.input):
            print("Success!")
        else:
            raise ValueError(
                "End of parse and these characters haven't been read: " + str(self.input[self.num_chars_read:]))

    ###########################################
    # Each nonterminal gets its own function. #
    ###########################################
    def E(self) -> None:
        # Production E--> N | OEE
        current_input = self.__get_current_input()
        if current_input in self.first['N']:
            self.N()
        elif current_input in self.first['O']:
            self.O()
            self.E()
            self.E()
        else:  # error
            raise ValueError("'E' can't produce input " + str(current_input))
    def N(self) -> None:
        # Production N --> 0 | 1 | 2 | 3
        current_input = self.__get_current_input()
        if self.__current_input_matches_target('0') or \
                self.__current_input_matches_target('1') or \
                self.__current_input_matches_target('2') or \
                self.__current_input_matches_target('3'):
            self.__advance_input()  # move to the next current input character
            pass
        else:
            raise ValueError("'N' can't produce input " + str(current_input))
    def O(self) -> None:
        # Production O --> + | *
        current_input = self.__get_current_input()
        if self.__current_input_matches_target('+') or \
                self.__current_input_matches_target('*'):
            self.__advance_input()  # move to the next current input character
            pass
        else:
            raise ValueError("'O' can't produce input " + str(current_input))

    ###################################################
    # Private helper functions for managing the input #
    ###################################################
    def __get_current_input(self) -> str:
        if self.num_chars_read == len(self.input):
            raise ValueError("Expected to read another input character but no inputs left to read")
        return self.input[self.num_chars_read]
    def __advance_input(self) -> None:
        if self.num_chars_read == len(self.input):
            raise ValueError("Expected to advance to next input character but no inputs left to read")
        self.num_chars_read += 1
    def __current_input_matches_target(self, target_input: str) -> bool:
        return self.__get_current_input() == target_input  

In [16]:
my_rdp: RDP = RDP()
try:
    my_rdp.parse_input('+12')
except ValueError as inst:
    message: tuple[str] = inst.args
    print(message)



('Expected to read another input character but no inputs left to read',)


In [13]:
my_rdp.reset()
try:
    my_rdp.parse_input('+123')
except ValueError as inst:
    message: tuple[str] = inst.args
    print(message)


Parsing string +123
			 In E function.
				 Trying to read input +
				 In O function.
					 Trying to read input +
					 In E function.
						 Trying to read input 1
						 In N function.
							 Trying to read input 1
						 Returning from E .
						 In E function.
							 Trying to read input 2
							 In N function.
								 Trying to read input 2
							 Returning from E .
						 Returning from E .
("End of parse and these characters haven't been read: 3",)


In [14]:
my_rdp.reset()
try:
    my_rdp.parse_input('+1')
except ValueError as inst:
    message: tuple[str] = inst.args
    print(message)

Parsing string +1
						 In E function.
							 Trying to read input +
							 In O function.
								 Trying to read input +
								 In E function.
									 Trying to read input 1
									 In N function.
										 Trying to read input 1
									 Returning from E .
									 In E function.
('Expected to read another input character but no inputs left to read',)


In [15]:
my_rdp.reset()
try:
    my_rdp.parse_input('+1A')
except ValueError as inst:
    message: tuple[str] = inst.args
    print(message)


Parsing string +1A
										 In E function.
											 Trying to read input +
											 In O function.
												 Trying to read input +
												 In E function.
													 Trying to read input 1
													 In N function.
														 Trying to read input 1
													 Returning from E .
													 In E function.
														 Trying to read input A
("Current input is A, which cannot be produced by 'E'",)


In [16]:
my_rdp.reset()
try:
    my_rdp.parse_input('-31')
except ValueError as inst:
    message: tuple[str] = inst.args
    print(message)

Parsing string -31
														 In E function.
															 Trying to read input -
															 In O function.
																 Trying to read input -
('Current input is -, which',)
